In [1]:
# 주요 라이브러리 import
import warnings
warnings.filterwarnings(action='ignore')
import time
from IPython.display import Image
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import *
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from sklearn import datasets
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso, ElasticNet

from matplotlib import rc, font_manager
import matplotlib.font_manager as fm
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

# 디렉토리 및 파일 이름에 맞추어 변경
font_location = '../../NanumGothic.ttf'
# font_location = './NanumGothic.ttf'
font_name = fm.FontProperties(fname=font_location).get_name()
rc('font', family=font_name)

In [2]:
df = pd.read_csv('data.csv') # 2019년 여행설문

In [3]:
df[:100]

,ID,Q,QQ,MON_EXP_1,MON_EXP_2,MON_EXP_3,MON_EXP_4,MON_EXP_5,MON_EXP_6,S5,...,국내_여행방문지_기타당일8,국내_여행방문지_기타당일9,국내_여행방문지_기타당일10,국내_여행방문지_기타당일11,국내_여행방문지_기타당일12,국내_여행방문지_기타당일13,국내_여행방문지_기타당일14,국내_여행방문지_기타당일15,국내_여행방문지_기타당일16,국내_여행방문지_기타당일17
0,11010550341_1000000,1,0,1,2,2,2,2,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11010550341_1000005,1,0,1,2,2,2,2,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11010550341_1000007,2,0,1,2,2,2,2,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11010550341_1000012,1,0,1,2,2,2,2,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11010550341_1000013,1,0,1,2,2,2,2,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,11020580301_1221834,1,0,1,2,2,2,2,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,11020580301_1221840,1,0,1,2,2,2,2,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,11020580301_1221847,1,0,1,2,2,2,2,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,11020580301_1221861,1,0,1,2,2,2,2,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
##input 년도에 여행한 사람만 추출하기 전 확인
year = 2019 ##년도
df.iloc[50:100,1:15][df['D_TRA1_SYEAR']==year]

,Q,QQ,MON_EXP_1,MON_EXP_2,MON_EXP_3,MON_EXP_4,MON_EXP_5,MON_EXP_6,S5,S5A_1,S5A_2,S5A_3,S5A_4,S5A_5
50,2,0,2,1,1,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN
53,2,0,1,2,2,2,2,1,NaN,NaN,NaN,NaN,NaN,NaN
55,1,0,2,2,1,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN
56,2,0,1,2,1,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN
58,2,0,1,2,1,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN
59,2,0,1,2,1,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN
60,1,0,1,2,2,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN
61,1,0,1,2,2,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN
62,1,0,2,2,2,2,1,2,NaN,NaN,NaN,NaN,NaN,NaN
63,1,0,1,2,2,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df['D_TRA1_SYEAR'].value_counts()

2019.0    27281
2018.0       40
Name: D_TRA1_SYEAR, dtype: int64

In [6]:
##year 년도에 여행한 사람만 추출
df = df[df['D_TRA1_SYEAR']==year] 


In [7]:
## 관광/휴양 이 포함되지 않는 row 제거
df = df[(df['MON_EXP_1']!=0) | (df['MON_EXP_2']!=0.0) | (df['MON_EXP_4']!=0)]

In [8]:
## 추천도, 재방문의사 보통이상만 추출
df = df[(df['A13']>=3) & (df['A14']>=3)]
df

,ID,Q,QQ,MON_EXP_1,MON_EXP_2,MON_EXP_3,MON_EXP_4,MON_EXP_5,MON_EXP_6,S5,...,국내_여행방문지_기타당일8,국내_여행방문지_기타당일9,국내_여행방문지_기타당일10,국내_여행방문지_기타당일11,국내_여행방문지_기타당일12,국내_여행방문지_기타당일13,국내_여행방문지_기타당일14,국내_여행방문지_기타당일15,국내_여행방문지_기타당일16,국내_여행방문지_기타당일17
1,11010550341_1000005,1,0,1,2,2,2,2,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11010550341_1000007,2,0,1,2,2,2,2,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11010550341_1000012,1,0,1,2,2,2,2,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11010550341_1000013,1,0,1,2,2,2,2,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,11010550341_1000018,1,0,1,2,2,2,2,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48229,39020600401_1319375,1,0,1,2,2,2,2,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48246,3902061022A_1382196,1,0,1,2,2,2,2,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48247,3902061022A_1382200,1,0,2,1,2,2,2,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48260,39020620091_1382242,1,0,1,2,2,2,2,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
## 1차 필요컬럼
columns_list = [
    'D_TRA1_SYEAR',
    'A3_1',
    'A3_2',
    'A3_3',
    'A3_4',
    'A3_5',
    'A3_6',
    'A3_7',
    'A3_8',
    'A3_9',
    'A3_10',
    'A3_11',
    'A3_12',
    'A3_13',
    'A3_14',
    'A3_15',
    'A3_16',
    'A3_17',
    'A3_18',
    'A3_19',
    'A3_20',
    'A3_21',
    'BSEX',
    'BAGE',
    'BAGE',
    'BMON',
    'BARA',
    'A12',
    'A10',
    'A7A',
    'A7',
    'A7B_1',
    'A7B_2',
    'A7B_3',
    'A7B_4',
    'A7B_5',
    'A7B_6',
    'A7B_7',
    'A2_1',
    'A2_2',
    'A2_3',
    'A5_1',
    'A5_2',
    'A5_3',
    'A6_1',
    'A6_2',
    'A6_3',
    'A6_4',
    'A6_5',
    'A6_6',
    'A6_7',
    'A6_8',
    'A6_9',
    'A6_10',
    'A6_11',
    'A6_12',
    'A6_13',
    'A6_14',
    'D_TRA1_SMONTH',
    'D_TRA2_SMONTH',
    'D_TRA3_SMONTH',
    'D_TRA4_SMONTH',
    'D_TRA5_SMONTH',
    'D_TRA6_SMONTH',
    'D_TRA7_SMONTH',
    'D_TRA8_SMONTH',
    'D_TRA1_1_SPOT1',
    'D_TRA1_1_SPOT2',
    'D_TRA1_1_SPOT3',
    'D_TRA1_1_SPOT4',
    'D_TRA1_1_SPOT5',
    'D_TRA1_1_SPOT6',
    'D_TRA1_1_SPOT7',
    'D_TRA1_1_SPOT8',
    'D_TRA1_1_SPOT9',
    'D_TRA1_1_SPOT10',
    'D_TRA2_1_SPOT1',
    'D_TRA2_1_SPOT2',
    'D_TRA2_1_SPOT3',
    'D_TRA2_1_SPOT4',
    'D_TRA2_1_SPOT5',
    'D_TRA2_1_SPOT6',
    'D_TRA2_1_SPOT7',
    'D_TRA2_1_SPOT8',
    'D_TRA2_1_SPOT9',
    'D_TRA2_1_SPOT10',
    'D_TRA3_1_SPOT1',
    'D_TRA3_1_SPOT2',
    'D_TRA3_1_SPOT3',
    'D_TRA3_1_SPOT4',
    'D_TRA3_1_SPOT5',
    'D_TRA3_1_SPOT6',
    'D_TRA3_1_SPOT7',
    'D_TRA3_1_SPOT8',
    'D_TRA3_1_SPOT9',
    'D_TRA3_1_SPOT10',
    'D_TRA4_1_SPOT1',
    'D_TRA4_1_SPOT2',
    'D_TRA4_1_SPOT3',
    'D_TRA4_1_SPOT4',
    'D_TRA4_1_SPOT5',
    'D_TRA4_1_SPOT6',
    'D_TRA4_1_SPOT7',
    'D_TRA4_1_SPOT8',
    'D_TRA4_1_SPOT9',
    'D_TRA4_1_SPOT10',
    'D_TRA5_1_SPOT1',
    'D_TRA5_1_SPOT2',
    'D_TRA5_1_SPOT3',
    'D_TRA5_1_SPOT4',
    'D_TRA5_1_SPOT5',
    'D_TRA5_1_SPOT6',
    'D_TRA5_1_SPOT7',
    'D_TRA5_1_SPOT8',
    'D_TRA5_1_SPOT9',
    'D_TRA5_1_SPOT10',
    'D_TRA6_1_SPOT1',
    'D_TRA6_1_SPOT2',
    'D_TRA6_1_SPOT3',
    'D_TRA6_1_SPOT4',
    'D_TRA6_1_SPOT5',
    'D_TRA6_1_SPOT6',
    'D_TRA6_1_SPOT7',
    'D_TRA6_1_SPOT8',
    'D_TRA6_1_SPOT9',
    'D_TRA6_1_SPOT10',
    'D_TRA7_1_SPOT1',
    'D_TRA7_1_SPOT2',
    'D_TRA7_1_SPOT3',
    'D_TRA7_1_SPOT4',
    'D_TRA7_1_SPOT5',
    'D_TRA7_1_SPOT6',
    'D_TRA7_1_SPOT7',
    'D_TRA7_1_SPOT8',
    'D_TRA7_1_SPOT9',
    'D_TRA7_1_SPOT10',
    'D_TRA8_1_SPOT1',
    'D_TRA8_1_SPOT2',
    'D_TRA8_1_SPOT3',
    'D_TRA8_1_SPOT4',
    'D_TRA8_1_SPOT5',
    'D_TRA8_1_SPOT6',
    'D_TRA8_1_SPOT7',
    'D_TRA8_1_SPOT8',
    'D_TRA8_1_SPOT9',
    'D_TRA8_1_SPOT10',
    'D_TRA1_S_Day',
    'D_TRA2_S_Day',
    'D_TRA3_S_Day',
    'D_TRA4_S_Day',
    'D_TRA5_S_Day',
    'D_TRA6_S_Day',
    'D_TRA7_S_Day',
    'D_TRA8_S_Day',
]
    
    

In [10]:
df = df[columns_list]

In [11]:
df.reset_index(inplace=True)
del df['index']
df

,D_TRA1_SYEAR,A3_1,A3_2,A3_3,A3_4,A3_5,A3_6,A3_7,A3_8,A3_9,...,D_TRA8_1_SPOT9,D_TRA8_1_SPOT10,D_TRA1_S_Day,D_TRA2_S_Day,D_TRA3_S_Day,D_TRA4_S_Day,D_TRA5_S_Day,D_TRA6_S_Day,D_TRA7_S_Day,D_TRA8_S_Day
0,2019.0,1.0,2.0,NaN,NaN,NaN,6.0,NaN,NaN,NaN,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019.0,1.0,2.0,3.0,NaN,NaN,6.0,7.0,8.0,9.0,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019.0,1.0,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019.0,1.0,2.0,3.0,NaN,NaN,6.0,NaN,NaN,NaN,...,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22567,2019.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22568,2019.0,NaN,2.0,NaN,NaN,NaN,6.0,NaN,8.0,9.0,...,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22569,2019.0,NaN,2.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22570,2019.0,NaN,NaN,NaN,NaN,5.0,6.0,NaN,NaN,NaN,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df[['A6_1', 'A6_2','A6_3','A6_4','A6_5','A6_6','A6_7','A6_8','A6_9','A6_10','A6_11','A6_12','A6_13','A6_14']]

,A6_1,A6_2,A6_3,A6_4,A6_5,A6_6,A6_7,A6_8,A6_9,A6_10,A6_11,A6_12,A6_13,A6_14
0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22568,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN
22569,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22570,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
## 퍼진 컬럼들 압축 과정

## TRA1_1, TRA1_2 등 모든 여행지 파생변수 area로 ,로 구분해서 입력, 
## 여행지 개수 count하여 no_of_place 컬럼에입력
# 1-1 1-2 1-3 1-4 1-5 ... 8-10
# [1,2,3,4,5,6]
sleepnights_list = [] #여행지에서 잔 밤수(박) ex) 1박 2일 = 1
cost_list = [] # 여행비용 <=이후 중복컬럼 생겨서 미사용
area_list = [] # 거주지 
count_list = [] # 여행지 개수 일치확인용 
month_list = [] # 여행 출발 월
for index in df.index:
    sleepnights = ''
    cost = ''
    area = ''
    count = 0
    month = ''
    # accommodation = ''
    for i in range (1,9):
        col_sleepnights = 'D_TRA'+str(i)+'_S_Day'       
        col_cost = 'D_TRA'+str(i)+'_COST'        
        try: 
            sleepnights = sleepnights + str(int(df.loc[index,col_sleepnights])) + ','
            cost = cost + str(int(df.loc[index,col_cost])) +','
        except: pass
        for j in range(1,11):

            col_area ='D_TRA'+str(i)+'_1'+'_SPOT'+str(j)
            col_month = 'D_TRA'+str(i)+'_'+str(j)+'_SMONTH'
            
            try: 
                area = area + str(int(df.loc[index, col_area])) + ','
                count += 1
            except: pass
            try:
                month = month + str(int(df.loc[index, col_month])) + ','
            except:
                pass
            # for k in range(1,15):
            #     col_accommodation = 'D_TRA'+str(i)+'_'+str(j)+'A6_'+str(k)
            #     try:
            #         accommodation = accommodation + str(int(df.loc[index, col_accommodation])) + ','
            #     except:
            #         pass
    try:
        area = area.rstrip(area[-1])
    except:pass
    try:
        sleepnights = sleepnights.rstrip(sleepnights[-1])
    except:pass
    try:
        cost = cost.rstrip(cost[-1])
    except:pass
    try:
        month = month.rstrip(month[-1])
    except:pass
    # try:
    #     accommodation = accommodation.rstrip(accommodation[-1])
    # except:pass
    area_list.append(area)
    count_list.append(count)
    month_list.append(month)
    cost_list.append(cost)
    # accommodation_list.append(accommodation)
    sleepnights_list.append(sleepnights)

df['area']= area_list
df['no_of_place'] = count_list
df['month'] = month_list
df['cost'] = cost_list
df['sleepnights'] = sleepnights_list
# df['accommodation'] = accommodation_list



In [14]:
accommodation_list = []
for index in df.index:
    accommodation = ''
    for k in range(1,15):
        col_accommodation = 'A6_'+str(k)
        try:
            accommodation = accommodation + str(int(df.loc[index, col_accommodation])) + ','
        except:
            pass
    try:
        accommodation = accommodation.rstrip(accommodation[-1])
    except:pass
    accommodation_list.append(accommodation)
df['accommodation'] = accommodation_list

In [15]:
df['accommodation']

0        2
1        9
2        3
3        4
4        7
        ..
22567     
22568    9
22569     
22570    1
22571    1
Name: accommodation, Length: 22572, dtype: object

In [16]:
## 확인
df[['area', 'no_of_place', 'month', 'cost', 'sleepnights', 'accommodation']]

,area,no_of_place,month,cost,sleepnights,accommodation
0,36460,1,,,2,2
1,21090,1,,,2,9
2,31370,1,,,2,3
3,39020,1,,,3,4
4,32060,1,,,1,7
...,...,...,...,...,...,...
22567,39010,1,,,0,
22568,11060,1,,,3,9
22569,39010,1,,,0,
22570,11210,1,,,2,1


In [17]:
## 2차 필요컬럼
# columns_list = [
#  'D_TRA1_SYEAR',
#  'area', 'no_of_place', 'cost', 'sleepnights', 'accommodation',
#  'A1_1',
#  'A1_2',
#  'A1_3',
#  'A4_1',
#  'A4_2',
#  'A4_3',
#  'A5_1',
#  'A5_2',
#  'A5_3',
#  'A5_4',
#  'A5_5',
#  'A5_6',
#  'A5_7',
#  'A5_8',
#  'A5_9',
#  'A5_10',
#  'A5_11',
#  'A5_12',
#  'A5_13',
#  'A5_14',
#  'A5_15',
#  'A5_16',
#  'A5_17',
#  'A5_18',
#  'A5_19',
#  'A5_20',
#  'A5_21',
#  'A7',
#  'A7A',
#  'A7B_1',
#  'A7B_2',
#  'A7B_3',
#  'A7B_4',
#  'A7B_5',
#  'A7B_6',
#  'A7B_7',
#  'A9',
#  'NA9',
#  'A11',
#  'A11A_1',
#  'A11A_2',
#  'A11A_3',
#  'A11A_4',
#  'A11A_5',
#  'A11A_6',
#  'A11A_7',
#  'A11A_8',
#  'A11A_9',
#  'A11A_10',
#  'A11A_11',
#  'A11A_12',
#  'DQ7',
#  'BSEX',
#  'SAGE',
#  'BMON',
#  'BARA'
# ]

In [18]:
# df = df[columns_list]

In [19]:
## 타겟 컬럼 list (미사용포함)
target_rawcol = []
for i in range(1,22):
    target_rawcol.append("A3_"+str(i))
target_rawcol

['A3_1',
 'A3_2',
 'A3_3',
 'A3_4',
 'A3_5',
 'A3_6',
 'A3_7',
 'A3_8',
 'A3_9',
 'A3_10',
 'A3_11',
 'A3_12',
 'A3_13',
 'A3_14',
 'A3_15',
 'A3_16',
 'A3_17',
 'A3_18',
 'A3_19',
 'A3_20',
 'A3_21']

In [20]:
# Target값 전처리

# Q5. 다녀오신 여행에서 참여하신 활동은 무엇입니까? 해당되는 항목은 모두 선택해 주십시오.
# column명: A5_1 ~ A5_21
# 전처리 전:
# 자연 및 풍경감상
# 음식관광(지역 맛집 등)
# 야외 위락 및 스포츠, 레포츠 활동
# 역사 유적지 방문
# 테마파크, 놀이시설, 동/식물원 방문
# 휴식/휴양
# 온천/스파
# 쇼핑
# 지역 문화예술/공연/전시시설 관람
# 스포츠 경기관람
# 지역 축제/이벤트참가
# 교육/체험 프로그램 참가
# 종교/성지순례
# 카지노, 경마, 경륜 등
# 시티투터
# 드라마 촬영지 방문
# 유흥/오락
# 가족/친지/친구 방문
# 회의참가/시찰
# 교육/훈련/연수
# 기타

# 전처리 후:
# 1.자연 및 풍경감상
# 2.음식관광(지역 맛집 등)
# 3.야외 위락 및 스포츠, 레포츠 활동
# 4.역사 유적지 방문                                  + 16. 드라마 촬영지 방문
# 5.테마파크, 놀이시설, 동/식물원 방문
# 6.휴식/휴양                                            + 7. 온천/스파
# 8.쇼핑                                                   + 15. 시티투어
# 9.지역 문화예술/공연/전시시설 관람         + 10.스포츠 경기관람, 11.지역 축제/이벤트참가 +12. 교육/체험 프로그램 참가
# 17.유흥/오락                                           +14.카지노, 경마, 경륜 등
# 21.기타

# 삭제할 컬럼
# 13.종교/성지순례
# 18.가족/친지/친구 방문
# 19.회의참가/시찰
# 20.교육/훈련/연수

In [21]:
# 로우 드랍 할 target
# 13.종교/성지순례 385
# 18.가족/친지/친구 방문 2755
# 19.회의참가/시찰 244
# 20.교육/훈련/연수 185
# 21.기타 89 합:4081
dropcol = ['A3_13','A3_18','A3_19','A3_20', 'A3_21']
intdropcol = [13,18,19,20,21]
## 안쓰는 target row drop (기준 밖 로우들 제거)
for i, col in enumerate(dropcol):
    df = df[df[col] != intdropcol[i]]
## 안쓰는 target column drop
df.drop(columns = dropcol, inplace=True)
df

,D_TRA1_SYEAR,A3_1,A3_2,A3_3,A3_4,A3_5,A3_6,A3_7,A3_8,A3_9,...,D_TRA5_S_Day,D_TRA6_S_Day,D_TRA7_S_Day,D_TRA8_S_Day,area,no_of_place,month,cost,sleepnights,accommodation
0,2019.0,1.0,2.0,NaN,NaN,NaN,6.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,36460,1,,,2,2
1,2019.0,1.0,2.0,3.0,NaN,NaN,6.0,7.0,8.0,9.0,...,NaN,NaN,NaN,NaN,21090,1,,,2,9
2,2019.0,1.0,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,31370,1,,,2,3
5,2019.0,1.0,2.0,NaN,NaN,NaN,6.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,36360,1,,,2,1
6,2019.0,NaN,2.0,NaN,NaN,NaN,NaN,7.0,NaN,NaN,...,NaN,NaN,NaN,NaN,25040,1,,,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22563,2019.0,1.0,2.0,NaN,NaN,NaN,6.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"39010,11230,39010,39010",4,,,"3,2,2,3",1
22566,2019.0,1.0,2.0,NaN,4.0,NaN,NaN,7.0,8.0,NaN,...,NaN,NaN,NaN,NaN,11230,1,,,13,"1,5,6"
22567,2019.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,39010,1,,,0,
22570,2019.0,NaN,NaN,NaN,NaN,5.0,6.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,11210,1,,,2,1


In [22]:
target_col = ['A3_1',
 'A3_2',
 'A3_3',
 'A3_4',
 'A3_5',
 'A3_6',
 'A3_7',
 'A3_8',
 'A3_9',
 'A3_10',
 'A3_11',
 'A3_12',
 'A3_14',
 'A3_15',
 'A3_16',
 'A3_17']
target_intlist = [1,2,3,4,5,6,7,8,9,10,11,12,14,15,16,17]
print(len(target_intlist), len(target_col))

16 16


In [23]:
##타겟 컬럼 압축 과정

##각 타겟 컬럼들안에 값이 있다면 추출, list에 저장
##새 컬럼에 list 삽입

target_list = []
for index in df.index:
    target_beforesplit=''
    for columns in target_col:
        try: 
            target_beforesplit = target_beforesplit + str(int(df.loc[index, columns])) + ','
        except:
            pass
    target_beforesplit = target_beforesplit.rstrip(target_beforesplit[-1])
    target_list.append(target_beforesplit)
target_list
df['target_beforesplit'] = target_list

In [24]:
## 기존 타겟 컬럼들 drop

## 타겟 컬럼들의 데이터가 압축되어 새 컬럼에 있으니
df.drop(columns=target_col, inplace=True)

In [25]:
## 교통수단 컬럼들 압축

transportaion_list = []
for index in df.index:
    transportation=''
    for columns in ['A5_1', 'A5_2', 'A5_3']:
        try: 
            transportation = transportation + str(int(df.loc[index, columns])) + ','
        except:
            pass
    transportation = transportation.rstrip(transportation[-1])
    transportaion_list.append(transportation)
transportaion_list
df['transportation'] = transportaion_list

In [26]:
## 교통 수단 압축 확인
df[['A5_1', 'A5_2', 'A5_3','transportation']].head(60)

,A5_1,A5_2,A5_3,transportation
0,2.0,4.0,NaN,"2,4"
1,2.0,5.0,3.0,"2,5,3"
2,1.0,NaN,NaN,1
5,1.0,NaN,NaN,1
6,1.0,NaN,NaN,1
7,1.0,NaN,NaN,1
9,1.0,NaN,NaN,1
11,1.0,NaN,NaN,1
12,1.0,NaN,NaN,1
13,6.0,4.0,3.0,"6,4,3"


In [27]:
#교통수단 컬럼들 제거
df.drop(columns=['A5_1', 'A5_2', 'A5_3'], inplace=True)


In [28]:
## 여행지 선택 이유 컬럼 압축
reason_list = []
for index in df.index:
    reason=''
    for columns in ['A2_1', 'A2_2', 'A2_3']:
        try: 
            reason = reason + str(int(df.loc[index, columns])) + ','
        except:
            pass
    reason = reason.rstrip(reason[-1])
    reason_list.append(reason)
transportaion_list
df['reason'] = reason_list

In [29]:
df.drop(columns=['A2_1', 'A2_2', 'A2_3'], inplace=True)

In [30]:
## 추가 미사용 컬럼 제거
delete_col = ['A7B_1',
 'A7B_2',
 'A7B_3',
 'A7B_4',
 'A7B_5',
 'A7B_6',
 'A7B_7']

In [31]:
## 여행자간 관계 컬럼 압축

relationship_list = []
for index in df.index:
    relationship=''
    for columns in delete_col:
        try: 
            relationship = relationship + str(int(df.loc[index, columns])) + ','
        except:
            pass
    try:
        relationship = relationship.rstrip(relationship[-1])
    except:pass
    relationship_list.append(relationship)
relationship_list
df['relationship_beforesplit'] = relationship_list
df[['A7B_1',
 'A7B_2',
 'A7B_3',
 'A7B_4',
 'A7B_5',
 'A7B_6',
 'A7B_7', 'relationship_beforesplit']].head(60)

,A7B_1,A7B_2,A7B_3,A7B_4,A7B_5,A7B_6,A7B_7,relationship_beforesplit
0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,3
1,NaN,NaN,3.0,NaN,NaN,NaN,NaN,3
2,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1
5,NaN,NaN,NaN,4.0,NaN,NaN,NaN,4
6,NaN,NaN,3.0,NaN,NaN,NaN,NaN,3
7,NaN,NaN,3.0,4.0,NaN,NaN,NaN,"3,4"
9,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1
11,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1
12,NaN,NaN,3.0,NaN,NaN,NaN,NaN,3
13,NaN,NaN,3.0,NaN,NaN,NaN,NaN,3


In [32]:
## 관계 컬럼 제거

df.drop(columns=delete_col, inplace=True)
df

,D_TRA1_SYEAR,BSEX,BAGE,BAGE,BMON,BARA,A12,A10,A7A,A7,...,area,no_of_place,month,cost,sleepnights,accommodation,target_beforesplit,transportation,reason,relationship_beforesplit
0,2019.0,1,5,5,1,11,4.0,250000.0,0.0,12.0,...,36460,1,,,2,2,"1,2,6","2,4","14,2,9",3
1,2019.0,1,1,1,1,11,4.0,217000.0,0.0,6.0,...,21090,1,,,2,9,"1,2,3,6,7,8,9","2,5,3","2,4,10",3
2,2019.0,1,4,4,1,11,4.0,348000.0,1.0,3.0,...,31370,1,,,2,3,"1,2,3",1,"9,1,4",1
5,2019.0,2,3,3,1,11,4.0,43000.0,0.0,3.0,...,36360,1,,,2,1,"1,2,6",1,"5,14,8",4
6,2019.0,2,2,2,1,11,4.0,200000.0,0.0,3.0,...,25040,1,,,0,,"2,7",1,12,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22563,2019.0,1,5,5,11,39,4.0,212000.0,0.0,5.0,...,"39010,11230,39010,39010",4,,,"3,2,2,3",1,"1,2,6,15",1,"14,1,8",1
22566,2019.0,2,3,3,9,39,5.0,1365000.0,0.0,2.0,...,11230,1,,,13,"1,5,6","1,2,4,7,8,11","3,10,2","14,5,7",3
22567,2019.0,1,5,5,10,39,3.0,100000.0,0.0,2.0,...,39010,1,,,0,,"1,2",1,"4,3",1
22570,2019.0,1,1,1,12,39,4.0,300000.0,0.0,12.0,...,11210,1,,,2,1,"5,6,12,15","3,6","13,14,5",6


In [33]:
# ## 추가 미사용 컬럼 제거

# df.drop(columns=[ 'A11A_1', 'A11A_2',
#        'A11A_3', 'A11A_4', 'A11A_5', 'A11A_6', 'A11A_7', 'A11A_8', 'A11A_9',
#        'A11A_10', 'A11A_11', 'A11A_12', 'DQ7'], inplace=True)

In [34]:
## 최종으로 남은 컬럼들 코드에서 명칭으로 컬러명 변경

df.rename(columns = {'D_TRA1_SYEAR' : 'year',
                     'A7' : 'number_of_ppl',
                     'A7A' : 'number_of_children',
                     'A10' : 'cost_total_trip',
                     'NA10': 'cost_total_per_person',
                     'A12' : 'satisfied',

                     'BSEX': 'sex', 
                     'BAGE': 'age', 
                     'BMON': 'month', 
                     'BARA': 'living_area',
                     'area': 'target_area'
                    
                     
                    }, inplace = True)



In [35]:
df.drop(columns='cost', inplace=True)

In [36]:
df = df.reset_index()
df

,index,year,sex,age,age,month,living_area,satisfied,cost_total_trip,number_of_children,...,D_TRA8_S_Day,target_area,no_of_place,month,sleepnights,accommodation,target_beforesplit,transportation,reason,relationship_beforesplit
0,0,2019.0,1,5,5,1,11,4.0,250000.0,0.0,...,NaN,36460,1,,2,2,"1,2,6","2,4","14,2,9",3
1,1,2019.0,1,1,1,1,11,4.0,217000.0,0.0,...,NaN,21090,1,,2,9,"1,2,3,6,7,8,9","2,5,3","2,4,10",3
2,2,2019.0,1,4,4,1,11,4.0,348000.0,1.0,...,NaN,31370,1,,2,3,"1,2,3",1,"9,1,4",1
3,5,2019.0,2,3,3,1,11,4.0,43000.0,0.0,...,NaN,36360,1,,2,1,"1,2,6",1,"5,14,8",4
4,6,2019.0,2,2,2,1,11,4.0,200000.0,0.0,...,NaN,25040,1,,0,,"2,7",1,12,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18141,22563,2019.0,1,5,5,11,39,4.0,212000.0,0.0,...,NaN,"39010,11230,39010,39010",4,,"3,2,2,3",1,"1,2,6,15",1,"14,1,8",1
18142,22566,2019.0,2,3,3,9,39,5.0,1365000.0,0.0,...,NaN,11230,1,,13,"1,5,6","1,2,4,7,8,11","3,10,2","14,5,7",3
18143,22567,2019.0,1,5,5,10,39,3.0,100000.0,0.0,...,NaN,39010,1,,0,,"1,2",1,"4,3",1
18144,22570,2019.0,1,1,1,12,39,4.0,300000.0,0.0,...,NaN,11210,1,,2,1,"5,6,12,15","3,6","13,14,5",6


In [37]:
del df['index']

In [38]:
df.to_csv("2019_압축버전.csv")

In [39]:
df

,year,sex,age,age,month,living_area,satisfied,cost_total_trip,number_of_children,number_of_ppl,...,D_TRA8_S_Day,target_area,no_of_place,month,sleepnights,accommodation,target_beforesplit,transportation,reason,relationship_beforesplit
0,2019.0,1,5,5,1,11,4.0,250000.0,0.0,12.0,...,NaN,36460,1,,2,2,"1,2,6","2,4","14,2,9",3
1,2019.0,1,1,1,1,11,4.0,217000.0,0.0,6.0,...,NaN,21090,1,,2,9,"1,2,3,6,7,8,9","2,5,3","2,4,10",3
2,2019.0,1,4,4,1,11,4.0,348000.0,1.0,3.0,...,NaN,31370,1,,2,3,"1,2,3",1,"9,1,4",1
3,2019.0,2,3,3,1,11,4.0,43000.0,0.0,3.0,...,NaN,36360,1,,2,1,"1,2,6",1,"5,14,8",4
4,2019.0,2,2,2,1,11,4.0,200000.0,0.0,3.0,...,NaN,25040,1,,0,,"2,7",1,12,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18141,2019.0,1,5,5,11,39,4.0,212000.0,0.0,5.0,...,NaN,"39010,11230,39010,39010",4,,"3,2,2,3",1,"1,2,6,15",1,"14,1,8",1
18142,2019.0,2,3,3,9,39,5.0,1365000.0,0.0,2.0,...,NaN,11230,1,,13,"1,5,6","1,2,4,7,8,11","3,10,2","14,5,7",3
18143,2019.0,1,5,5,10,39,3.0,100000.0,0.0,2.0,...,NaN,39010,1,,0,,"1,2",1,"4,3",1
18144,2019.0,1,1,1,12,39,4.0,300000.0,0.0,12.0,...,NaN,11210,1,,2,1,"5,6,12,15","3,6","13,14,5",6


# 카테고리 전처리 ==========================================================================

In [40]:
df

,year,sex,age,age,month,living_area,satisfied,cost_total_trip,number_of_children,number_of_ppl,...,D_TRA8_S_Day,target_area,no_of_place,month,sleepnights,accommodation,target_beforesplit,transportation,reason,relationship_beforesplit
0,2019.0,1,5,5,1,11,4.0,250000.0,0.0,12.0,...,NaN,36460,1,,2,2,"1,2,6","2,4","14,2,9",3
1,2019.0,1,1,1,1,11,4.0,217000.0,0.0,6.0,...,NaN,21090,1,,2,9,"1,2,3,6,7,8,9","2,5,3","2,4,10",3
2,2019.0,1,4,4,1,11,4.0,348000.0,1.0,3.0,...,NaN,31370,1,,2,3,"1,2,3",1,"9,1,4",1
3,2019.0,2,3,3,1,11,4.0,43000.0,0.0,3.0,...,NaN,36360,1,,2,1,"1,2,6",1,"5,14,8",4
4,2019.0,2,2,2,1,11,4.0,200000.0,0.0,3.0,...,NaN,25040,1,,0,,"2,7",1,12,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18141,2019.0,1,5,5,11,39,4.0,212000.0,0.0,5.0,...,NaN,"39010,11230,39010,39010",4,,"3,2,2,3",1,"1,2,6,15",1,"14,1,8",1
18142,2019.0,2,3,3,9,39,5.0,1365000.0,0.0,2.0,...,NaN,11230,1,,13,"1,5,6","1,2,4,7,8,11","3,10,2","14,5,7",3
18143,2019.0,1,5,5,10,39,3.0,100000.0,0.0,2.0,...,NaN,39010,1,,0,,"1,2",1,"4,3",1
18144,2019.0,1,1,1,12,39,4.0,300000.0,0.0,12.0,...,NaN,11210,1,,2,1,"5,6,12,15","3,6","13,14,5",6


In [41]:
## target값 split하여 One-hot encoding
for index in df.index:
  target_list_index = df.loc[index,'target_beforesplit'].split(',')
  for targets in target_list_index:
    for i in [1,2,3,4,5,6,7,8,9,10,11,12,14,15,16,17]:
      t_col = 'target_'+str(i)
      if targets == str(i):
        df.loc[index,t_col]= 1

In [42]:
df[['target_beforesplit', 'target_1', 'target_2', 'target_3',
       'target_4', 'target_5', 'target_6', 'target_7', 'target_8', 'target_9',
       'target_10', 'target_11', 'target_12', 'target_14', 'target_15',
       'target_16', 'target_17']]

,target_beforesplit,target_1,target_2,target_3,target_4,target_5,target_6,target_7,target_8,target_9,target_10,target_11,target_12,target_14,target_15,target_16,target_17
0,"1,2,6",1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"1,2,3,6,7,8,9",1.0,1.0,1.0,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"1,2,3",1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"1,2,6",1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"2,7",NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18141,"1,2,6,15",1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
18142,"1,2,4,7,8,11",1.0,1.0,NaN,1.0,NaN,NaN,1.0,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
18143,"1,2",1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18144,"5,6,12,15",NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN


In [43]:
df

,year,sex,age,age,month,living_area,satisfied,cost_total_trip,number_of_children,number_of_ppl,...,target_9,target_16,target_4,target_11,target_5,target_15,target_17,target_12,target_14,target_10
0,2019.0,1,5,5,1,11,4.0,250000.0,0.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019.0,1,1,1,1,11,4.0,217000.0,0.0,6.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019.0,1,4,4,1,11,4.0,348000.0,1.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019.0,2,3,3,1,11,4.0,43000.0,0.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019.0,2,2,2,1,11,4.0,200000.0,0.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18141,2019.0,1,5,5,11,39,4.0,212000.0,0.0,5.0,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
18142,2019.0,2,3,3,9,39,5.0,1365000.0,0.0,2.0,...,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
18143,2019.0,1,5,5,10,39,3.0,100000.0,0.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18144,2019.0,1,1,1,12,39,4.0,300000.0,0.0,12.0,...,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1.0,NaN,NaN


In [44]:
## relationship값 split하여 One-hot encoding
for index in df.index:
  target_list_index = df.loc[index,'relationship_beforesplit'].split(',')
  for i, targets in enumerate(target_list_index):
      t_col = 'relationship'+str(i+1)
      df.loc[index,t_col]= targets

In [45]:
## 여행지 장소 split
for index in df.index:
  target_list_index = df.loc[index,'target_area'].split(',')
  for i, targets in enumerate(target_list_index):
      t_col = 'target_area'+str(i+1)
      df.loc[index,t_col]= targets

In [46]:
## 14박 이상 잔 row제거
for i in df.index: 
    if sum(list(map(int, df.loc[index, 'sleepnights'].split(',')))) > 14:
        df.drop(index=i, inplace=True)

In [47]:
## 잔 횟수와(박)과 숙소 일치시키기

## 반복된 숙소 중복제거후 sleepnight당 나올 수 있는 
## accommodation 경우의수 마다 조건으로 처리
## 마지막 예외 제거 : 4row

## 반복된 숫자 줄이는 메서드
def shrink_accom(accom): 
    i = 0
    result = accom
    while i < len(result)-1:
        if result[i] == result[i+1]:
            del result[i]
        else:
            i = i+1
    return result

In [48]:
df.loc[:50,'accommodation']

0      2
1      9
2      3
3      1
4       
5      3
6      4
7      2
8     13
9      9
10      
11    10
12     4
13     3
14      
15     1
16     2
17    12
18      
19      
20      
21     1
22      
23      
24     3
25      
26     2
27      
28      
29     3
30      
31     4
32      
33      
34    12
35     4
36      
37     3
38     3
39      
40    11
41     3
42    11
43     3
44     1
45      
46     8
47      
48     1
49    10
50    11
Name: accommodation, dtype: object

In [49]:
test = []
for i in range(1,15):
    test.append('A6_'+str(i))
test

['A6_1',
 'A6_2',
 'A6_3',
 'A6_4',
 'A6_5',
 'A6_6',
 'A6_7',
 'A6_8',
 'A6_9',
 'A6_10',
 'A6_11',
 'A6_12',
 'A6_13',
 'A6_14']

In [50]:
df[test]

,A6_1,A6_2,A6_3,A6_4,A6_5,A6_6,A6_7,A6_8,A6_9,A6_10,A6_11,A6_12,A6_13,A6_14
0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18141,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18142,1.0,NaN,NaN,NaN,5.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18143,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18144,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
df = df[df['accommodation'].notna()]


In [52]:
df[['sleepnights','accommodation']].head(60)

,sleepnights,accommodation
0,2,2
1,2,9
2,2,3
3,2,1
4,0,
5,1,3
6,1,4
7,1,2
8,1,13
9,2,9


In [53]:
df.to_csv('test.csv')

In [54]:
df = pd.read_csv('test.csv', index_col=0)

In [55]:
df = df[df['accommodation']!='']
df

,year,sex,age,age.1,month,living_area,satisfied,cost_total_trip,number_of_children,number_of_ppl,...,relationship3,relationship4,target_area1,target_area2,target_area3,target_area4,target_area5,target_area6,target_area7,target_area8
0,2019.0,1,5,5,1,11,4.0,250000.0,0.0,12.0,...,NaN,NaN,36460,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019.0,1,1,1,1,11,4.0,217000.0,0.0,6.0,...,NaN,NaN,21090,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019.0,1,4,4,1,11,4.0,348000.0,1.0,3.0,...,NaN,NaN,31370,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019.0,2,3,3,1,11,4.0,43000.0,0.0,3.0,...,NaN,NaN,36360,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019.0,2,2,2,1,11,4.0,200000.0,0.0,3.0,...,NaN,NaN,25040,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18141,2019.0,1,5,5,11,39,4.0,212000.0,0.0,5.0,...,NaN,NaN,39010,11230.0,39010.0,39010.0,NaN,NaN,NaN,NaN
18142,2019.0,2,3,3,9,39,5.0,1365000.0,0.0,2.0,...,NaN,NaN,11230,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18143,2019.0,1,5,5,10,39,3.0,100000.0,0.0,2.0,...,NaN,NaN,39010,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18144,2019.0,1,1,1,12,39,4.0,300000.0,0.0,12.0,...,NaN,NaN,11210,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
df = df.astype('object')

In [57]:
df.dtypes[100:150]

D_TRA7_1_SPOT9              object
D_TRA7_1_SPOT10             object
D_TRA8_1_SPOT1              object
D_TRA8_1_SPOT2              object
D_TRA8_1_SPOT3              object
D_TRA8_1_SPOT4              object
D_TRA8_1_SPOT5              object
D_TRA8_1_SPOT6              object
D_TRA8_1_SPOT7              object
D_TRA8_1_SPOT8              object
D_TRA8_1_SPOT9              object
D_TRA8_1_SPOT10             object
D_TRA1_S_Day                object
D_TRA2_S_Day                object
D_TRA3_S_Day                object
D_TRA4_S_Day                object
D_TRA5_S_Day                object
D_TRA6_S_Day                object
D_TRA7_S_Day                object
D_TRA8_S_Day                object
target_area                 object
no_of_place                 object
month.1                     object
sleepnights                 object
accommodation               object
target_beforesplit          object
transportation              object
reason                      object
relationship_befores

In [58]:
df[df['accommodation']==None]

,year,sex,age,age.1,month,living_area,satisfied,cost_total_trip,number_of_children,number_of_ppl,...,relationship3,relationship4,target_area1,target_area2,target_area3,target_area4,target_area5,target_area6,target_area7,target_area8


In [59]:
## 잔 횟수와(박)과 숙소 일치시키기

## 반복된 숙소 중복제거후 sleepnight당 나올 수 있는 
## accommodation 경우의수 마다 조건으로 처리
## 마지막 예외 제거 : 4row

## 반복된 숫자 줄이는 메서드
def shrink_accom(accom): 
    i = 0
    result = accom
    while i < len(result)-1:
        if result[i] == result[i+1]:
            del result[i]
        else:
            i = i+1
    return result

In [63]:
for index in df.index: ##당일여행 한번 갔을때: accommodation=0
    accommo_list = []
    try:splitted_nights = list(map(int, df.loc[index, 'sleepnights'].split(','))) ## sleepnights를 int로 split
    except:print("typeerror>>",index)
    if splitted_nights == [0]:
        df.loc[index, 'accommodation'] = 0
    else:
        s = np.count_nonzero(splitted_nights) ## splitted_nights중 0이 아닌 수 count s
        if s == 0: ## 0,0,0 와 같이 0만 있을 때, 0으로 채움            for i in range(len(splitted_nights)):
            for i in range(len(splitted_nights)):
                accommo_list.append(0)
            for i, accommo in enumerate(accommo_list):
                col_name = 'accommodation_' + str(i+1) ##dataframe에 입력
                df.loc[index,col_name] = 0
        else:
            try:splitted_accommo = list(map(int, df.loc[index, 'accommodation'].split(',')))
            except:
                print('delete FLOAT>>',index, df.loc[index,'sleepnights'], df.loc[index,'accommodation'], )
                df.drop(index=index, inplace=True)
                continue
            try:shrinked_accommo = shrink_accom(list(map(int, df.loc[index, 'accommodation'].split(',')))) ## 반복된 숫자 줄여줌
            except:print('FLOAT>>',index, df.loc[index,'sleepnights'], df.loc[index,'accommodation'])
            if (s == len(shrinked_accommo)) or (len(shrinked_accommo) == 1):## s와 accommo의 길이가 같을때: s의 수대로 accommo를 곱해준다
                if [len(shrinked_accommo)] == splitted_nights: ## 0미포함
                    for i, accommo in enumerate(shrinked_accommo):
                        col_name = 'accommodation_' + str(i+1) ##dataframe에 입력
                        df.loc[index,col_name] = accommo
                else:    ##0포함
                    j=0
                    for i, nights in enumerate(splitted_nights):
                        if nights == 0:
                            accommo_list.append(0)
                        else:
                            for ni in range(nights):
                                accommo_list.append(shrinked_accommo[j])
                    j+=1
                    for i, accommo in enumerate(accommo_list):
                        col_name = 'accommodation_' + str(i+1) ##dataframe에 입력
                        df.loc[index,col_name] = accommo
            elif s > len(shrinked_accommo): ## s가 accommo의 길이보다 클 때
                if len(list(map(int, df.loc[index, 'accommodation'].split(',')))) == len(splitted_nights):
                    for i, accommo in enumerate(splitted_nights):
                        col_name = 'accommodation_' + str(i+1) ##dataframe에 입력
                        df.loc[index,col_name] = accommo
                    
                else:
                    for i, nights in enumerate(splitted_nights): ## 3,0,1 12,0,1 => 12,12,12,0,1
                        if nights == 0:
                            accommo_list.append(0)
                        else:
                            for ni in range(nights):
                                try:accommo_list.append(shrinked_accommo[i])
                                except:print('indexError>>', index, "s:", s, "s_night:", splitted_nights , 
                                             "slpnight:", df.loc[index, 'sleepnights'], 
                                             "accom", df.loc[index, 'accommodation'], 
                                             "splt_accom:", splitted_accommo)
                    for i, accommo in enumerate(accommo_list):
                        col_name = 'accommodation_' + str(i+1) ##dataframe에 입력
                        df.loc[index,col_name] = accommo 
                
            else: 
                if sum(list(map(int, df.loc[1215, 'sleepnights'].split(',')))) == len(list(map(
                    int, df.loc[index, 'accommodation'].split(',')))): ## [0,0,2] [1,2]
                    j=0
                    for i, nights in enumerate(list(map(int, df.loc[1215, 'sleepnights'].split(',')))): 
                        if nights == 0:
                            accommo_list.append(0)
                        else:
                            for ni in range(nights):
                                accommo_list.append(list(map(int, df.loc[index, 'accommodation'].split(',')))[j])
                                j+=1
                    for i, accommo in enumerate(accommo_list):
                        col_name = 'accommodation_' + str(i+1) ##dataframe에 입력
                        df.loc[index,col_name] = accommo 
                if [len(shrinked_accommo)] == splitted_nights: ## 0미포함
                    for i, accommo in enumerate(shrinked_accommo):
                        col_name = 'accommodation_' + str(i+1) ##dataframe에 입력
                        df.loc[index,col_name] = accommo
                elif sum(list(map(int, df.loc[index, 'sleepnights'].split(',')))) == len(list(map(int, df.loc[index, 'accommodation'].split(',')))):
                    accommo_list = list(map(int, df.loc[index, 'accommodation'].split(',')))
                    for i, accommo in enumerate(accommo_list):
                        col_name = 'accommodation_' + str(i+1) ##dataframe에 입력
                        df.loc[index,col_name] = accommo
                    # print("caught row>>", index, "s:", s, "s_night:", splitted_nights , "slpnight:", df.loc[index, 'sleepnights'], "accom", df.loc[index, 'accommodation'], "shrink_accom:", shrinked_accommo)
                else:
                    print("drop row>>", index, "s:", s, "s_night:", splitted_nights , "slpnight:", df.loc[index, 'sleepnights'], "accom", df.loc[index, 'accommodation'], "shrink_accom:", shrinked_accommo)
                    df.drop(index=index, inplace=True)
                



delete FLOAT>> 28 0,2 nan
delete FLOAT>> 30 0,1 nan
delete FLOAT>> 80 2,0 nan
delete FLOAT>> 86 0,1 nan
delete FLOAT>> 134 2,0 nan
delete FLOAT>> 164 0,3 nan
delete FLOAT>> 335 0,3 nan
delete FLOAT>> 544 0,1 nan
delete FLOAT>> 646 0,1,0 nan
delete FLOAT>> 660 2,0 nan
delete FLOAT>> 664 1,0 nan
delete FLOAT>> 675 2,0 nan
delete FLOAT>> 690 1,0 nan
delete FLOAT>> 738 1,0 nan
delete FLOAT>> 748 1,0 nan
delete FLOAT>> 780 1,0 nan
delete FLOAT>> 805 1,1,0 nan
delete FLOAT>> 810 1,0 nan
delete FLOAT>> 841 1,0 nan
delete FLOAT>> 866 1,0 nan
delete FLOAT>> 918 0,1 nan
delete FLOAT>> 932 2,0 nan
delete FLOAT>> 1172 2,0 nan
delete FLOAT>> 1334 2,0 nan
delete FLOAT>> 1501 1,0 nan
delete FLOAT>> 1599 2,0 nan
delete FLOAT>> 1603 0,1 nan
delete FLOAT>> 1636 1,0 nan
delete FLOAT>> 1652 1,0 nan
delete FLOAT>> 1664 2,0 nan
delete FLOAT>> 1741 1,0 nan
delete FLOAT>> 1794 0,2 nan
delete FLOAT>> 1853 1,0 nan
delete FLOAT>> 1877 2,0 nan
delete FLOAT>> 1891 1,0 nan
delete FLOAT>> 1908 4,0,0,0 nan
delete FLO

In [64]:
for index in df.index:
  target_list_index = df.loc[index,'transportation'].split(',')
  for i, targets in enumerate(target_list_index):
       t_col = 'transportation'+str(i+1)
       df.loc[index,t_col]= targets

In [65]:
for index in df.index:
  target_list_index = df.loc[index,'reason'].split(',')
  for i, targets in enumerate(target_list_index):
       t_col = 'reason'+str(i+1)
       df.loc[index,t_col]= targets

In [66]:
df.columns[50:150]

Index(['D_TRA2_1_SPOT9', 'D_TRA2_1_SPOT10', 'D_TRA3_1_SPOT1', 'D_TRA3_1_SPOT2',
       'D_TRA3_1_SPOT3', 'D_TRA3_1_SPOT4', 'D_TRA3_1_SPOT5', 'D_TRA3_1_SPOT6',
       'D_TRA3_1_SPOT7', 'D_TRA3_1_SPOT8', 'D_TRA3_1_SPOT9', 'D_TRA3_1_SPOT10',
       'D_TRA4_1_SPOT1', 'D_TRA4_1_SPOT2', 'D_TRA4_1_SPOT3', 'D_TRA4_1_SPOT4',
       'D_TRA4_1_SPOT5', 'D_TRA4_1_SPOT6', 'D_TRA4_1_SPOT7', 'D_TRA4_1_SPOT8',
       'D_TRA4_1_SPOT9', 'D_TRA4_1_SPOT10', 'D_TRA5_1_SPOT1', 'D_TRA5_1_SPOT2',
       'D_TRA5_1_SPOT3', 'D_TRA5_1_SPOT4', 'D_TRA5_1_SPOT5', 'D_TRA5_1_SPOT6',
       'D_TRA5_1_SPOT7', 'D_TRA5_1_SPOT8', 'D_TRA5_1_SPOT9', 'D_TRA5_1_SPOT10',
       'D_TRA6_1_SPOT1', 'D_TRA6_1_SPOT2', 'D_TRA6_1_SPOT3', 'D_TRA6_1_SPOT4',
       'D_TRA6_1_SPOT5', 'D_TRA6_1_SPOT6', 'D_TRA6_1_SPOT7', 'D_TRA6_1_SPOT8',
       'D_TRA6_1_SPOT9', 'D_TRA6_1_SPOT10', 'D_TRA7_1_SPOT1', 'D_TRA7_1_SPOT2',
       'D_TRA7_1_SPOT3', 'D_TRA7_1_SPOT4', 'D_TRA7_1_SPOT5', 'D_TRA7_1_SPOT6',
       'D_TRA7_1_SPOT7', 'D_TRA7_1_SPOT8', 'D_T

In [70]:
## 컬럼 순서 정리

column_order = ['year', 'target_area', 'no_of_place', 'sleepnights', 'accommodation',
       'number_of_ppl', 'number_of_children', 'cost_total_trip',
       'cost_total_per_person', 'satisfied', 'sex', 'age', 'month',
       'living_area', 'target_beforesplit', 'transportation', 'reason',
       'relationship_beforesplit','relationship1', 'relationship2',
       'relationship3','target_1', 'target_2', 'target_3','target_4','target_5','target_6', 'target_7','target_8', 'target_9','target_10','target_11', 'target_12','target_14', 'target_15','target_16', 'target_17', 
        'target_area1', 'target_area2', 'target_area3',
       'target_area4', 'target_area5', 'target_area6', 'target_area7',
       'target_area8', 'target_area9', 'target_area10', 'target_area11',
       'transportation1', 'transportation2', 'transportation3', 'reason1',
       'reason2', 'reason3', 'accommodation_1', 'accommodation_2',
       'accommodation_3', 'accommodation_4', 'accommodation_5',
       'accommodation_6', 'accommodation_7', 'accommodation_8',
       'accommodation_9', 'accommodation_10', 'accommodation_11',
       'accommodation_12', 'accommodation_13', 'accommodation_14']



In [71]:
df = df.reindex(columns=column_order)

In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17636 entries, 0 to 18145
Data columns (total 68 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   year                      17636 non-null  object 
 1   target_area               17636 non-null  object 
 2   no_of_place               17636 non-null  object 
 3   sleepnights               17636 non-null  object 
 4   accommodation             16754 non-null  object 
 5   number_of_ppl             17636 non-null  object 
 6   number_of_children        17207 non-null  object 
 7   cost_total_trip           17636 non-null  object 
 8   cost_total_per_person     0 non-null      float64
 9   satisfied                 17636 non-null  object 
 10  sex                       17636 non-null  object 
 11  age                       17636 non-null  object 
 12  month                     17636 non-null  object 
 13  living_area               17636 non-null  object 
 14  target

In [73]:
df

,year,target_area,no_of_place,sleepnights,accommodation,number_of_ppl,number_of_children,cost_total_trip,cost_total_per_person,satisfied,...,accommodation_5,accommodation_6,accommodation_7,accommodation_8,accommodation_9,accommodation_10,accommodation_11,accommodation_12,accommodation_13,accommodation_14
0,2019.0,36460,1,2,2,12.0,0.0,250000.0,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019.0,21090,1,2,9,6.0,0.0,217000.0,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019.0,31370,1,2,3,3.0,1.0,348000.0,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019.0,36360,1,2,1,3.0,0.0,43000.0,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019.0,25040,1,0,0,3.0,0.0,200000.0,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18140,2019.0,11030,1,1,1,20.0,0.0,300000.0,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18141,2019.0,"39010,11230,39010,39010",4,"3,2,2,3",1,5.0,0.0,212000.0,NaN,4.0,...,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN
18143,2019.0,39010,1,0,0,2.0,0.0,100000.0,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18144,2019.0,11210,1,2,1,12.0,0.0,300000.0,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
df.to_csv('2019_split버전.csv')